In [ ]:
#goal: have a table that looks like this
#how to 

'''
|country|first_idx|second_idx|third_idx|

switzerland 1       13         12



////

then incorporate weights



tables DONE:
* salary (242) (biggest list - treated as master)
* safety (163); 92 404s? need to inspect
* life expectancy (183)
* days off (194)

to-do:


------Q------

need to do manual inspection for some countries (e.g. safety - Luxembourg)



----Notes----
NaN replaced with 404







In [1]:
import pandas as pd

sal = pd.read_csv("salary.csv", delimiter = ';', header = None, index_col = False)
sal.rename(columns = {0: 'salary idx', 1: 'Country', 2: 'Salary', 3: 'Currency', 4: 'Salary[USD]'}, inplace = True)
sal.sort_values(by = 'Salary[USD]', ascending = False, inplace = True)
sal.reset_index(drop=True, inplace=True)  
sal.tail()
sal.dropna(inplace = True)

countries = []
for country in sal['Country']:
    countries.append(country)

master = pd.DataFrame(columns = ["Country", "Salary", "Safety", "Life Expectancy", "Days Off"])
master.head()
master["Country"] = countries

master.head()



#sal.drop(['NaN'])

,Country,Salary,Safety,Life Expectancy,Days Off
0,Switzerland,NaN,NaN,NaN,NaN
1,United States,NaN,NaN,NaN,NaN
2,Denmark,NaN,NaN,NaN,NaN
3,Japan,NaN,NaN,NaN,NaN
4,Luxembourg,NaN,NaN,NaN,NaN


In [2]:
def master_add(column, listname = []):
    print column
    
    for country in countries:
        try:
            value = listname[listname['Country']==country].index.item()
            value+=1
        except: #Country not found
            #print "error"
            value = 404 #bc not found
        
        m_val = master[master['Country'] == country].index.item()   #not in the try block bc master has all the countries
        master.at[m_val, column] = value #doesn't work?

#    print m_val, column, value
    
    print master.at[1, column]
  #  return master
    

In [11]:
master_add("Salary", sal)

Salary
2


In [12]:
print master.head()

         Country Salary Safety Life Expectancy Days Off
0    Switzerland      1     10             NaN       95
1  United States      2    128             NaN      188
2        Denmark      3      5             NaN       23
3          Japan      4      9             NaN      185
4     Luxembourg      5    404             NaN        7


In [4]:
#already sorted

saf = pd.read_csv("scrapers/safety.csv", delimiter = ';', header = None, index_col = False)
saf.rename(columns = {0: 'safety idx', 1: 'Country', 2: 'Score'}, inplace = True)
saf.drop(columns = "Score", axis = 1, inplace = True)

saf.head()
#saf.head()

,safety idx,Country
0,1,Iceland
1,2,New Zealand
2,3,Portugal
3,4,Austria
4,5,Denmark


In [5]:
master_add("Safety", saf)

Safety
128


In [6]:
#already sorted

life_exp = pd.read_csv("scrapers/life-exp.csv", delimiter = ';', header = None, index_col = False)
life_exp.rename(columns = {0: 'life exp idx', 1: 'Country', 2: 'Predicted Score'}, inplace = True)
life_exp.drop(columns = "Predicted Score", inplace = True)
life_exp.head()



,life exp idx,Country
0,1,Japan
1,2,Switzerland
2,3,Singapore
3,4,Australia
4,5,Spain


In [13]:
master_add("Life Expectancy", life_exp)

Life Expectancy
31


In [7]:
days_off = pd.read_csv("scrapers/days_off.csv", delimiter = ';', header = None, index_col = False)
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)



days_off.head()

len(days_off)

194

In [8]:
master_add("Days Off", days_off)

Days Off
188


In [11]:
days_off.iloc[[50]]
days_off[days_off['Country']=="United States"].index.item() 
try:
    val = days_off[days_off['Country']=="United Kingdom"].index.item()
    print val
except: # country not in the list
    print "na"
        


83


In [29]:
lan = pd.read_csv("scrapers/languages.csv", delimiter = ';', header = None, index_col = False)

'''
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)

'''

lan.head()


#this is going to be used to determine whether the country speaks your language
# move to a later phase


,0,1,2
0,1,Albania,"Albanian (Tosk is the official dialect), Greek"
1,2,Algeria,"Arabic (official), French, Berber dialects"
2,3,Andorra,"Catalán (official), French, Castilian, Portuguese"
3,4,Angola,"Portuguese (official), Bantu and other African..."
4,5,Antigua and Barbuda,"English (official), local dialects"


In [30]:
taxes = pd.read_csv("scrapers/taxes.csv", delimiter = ';', header = None, index_col = False)
taxes.reset_index(drop = True, inplace = True)
taxes.sort_values(by = 2, ascending = True, inplace = True)
taxes.drop(columns = 0, inplace = True)
taxes.rename(columns = {1: "Country", 2: "Income Tax"}, inplace = True)
taxes.head()

,Country,Income Tax
156,United Arab Emirates,0.0
147,Bahamas,0.0
148,Bahrain,0.0
149,Bermuda,0.0
150,Brunei,0.0


In [38]:
master_add("Income Tax", taxes)

Income Tax
43.0


In [37]:
taxes.sort_values(by = "Income Tax", ascending = True)

,Country,Income Tax
156,United Arab Emirates,0.00
147,Bahamas,0.00
148,Bahrain,0.00
149,Bermuda,0.00
150,Brunei,0.00
155,Saudi Arabia,0.00
152,Kuwait,0.00
153,Oman,0.00
154,Qatar,0.00
151,Cayman Islands,0.00


In [39]:
master.head()
#wrong value for Denmark - idk why
# and wrong value for the US in days off

# look into the master_add - doesn't seem to be working correctly


,Country,Salary,Safety,Life Expectancy,Days Off,Income Tax
0,Switzerland,1,10,2,95,35.0
1,United States,2,128,31,188,43.0
2,Denmark,3,5,27,23,6.0
3,Japan,4,9,1,185,5.0
4,Luxembourg,5,404,13,7,14.0


In [ ]:
'''
ideas for distribution:
1. points
    100 points in total
    by choosing which category is more important -> that category gets more points
    then in each category can assign the points by choice:
            eg. if max 10 points to distribute
                    1-10: 10
                    10-20: 8
                    
                    .. etc (Nan's get 0 - but need to make sure which ones are actually NaN - manual inspection)
                    
                    
                    

next phases of work:
    * reading the remaining csv's
    OR * experimenting with the basic functionality




'''

In [15]:
#countring the ones that are 404
saf_null = master[master["Safety"] == 404].count()
life_exp_null = master[master["Life Expectancy"] == 404].count()
days_off_null = master[master["Days Off"] == 404].count()

print saf_null, life_exp_null, days_off_null


Country            92
Salary             92
Safety             92
Life Expectancy    92
Days Off           92
dtype: int64 Country            68
Salary             68
Safety             68
Life Expectancy    68
Days Off           68
dtype: int64 Country            61
Salary             61
Safety             61
Life Expectancy    61
Days Off           61
dtype: int64
